## 데이터 파싱 코드

In [ ]:
pip install -qU langchain_community

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

# CSV 파일 경로
loader = CSVLoader(
    file_path="/content/data/boston_housing.csv",
    csv_args={
        "delimiter": ",",  # 구분자
        "quotechar": '"',  # 인용 부호 문자
        "fieldnames": [
            "CRIM",   # 인구당 범죄율
            "ZN",     # 25,000 평방피트를 초과하는 거주 구역 비율
            "INDUS",  # 비소매 상업 지역 비율
            "CHAS",   # 찰스강 인접 여부 (1: 인접, 0: 비인접)
            "NOX",    # 일산화질소 농도
            "RM",     # 주택당 평균 방 개수
            "AGE",    # 1940년 이전에 건축된 주택 비율
            "DIS",    # 5개 보스턴 고용 센터까지의 거리
            "RAD",    # 고속도로 접근성 지수
            "TAX",    # 10,000달러당 재산세율
            "PTRATIO", # 지역별 학생-교사 비율
            "B",      # 흑인 비율
            "LSTAT",  # 저소득층 비율
            "PRICE",  # 주택 가격 (목표 변수)
        ],  # 필드 이름
    },
)


## UNSTRUCTED

In [ ]:
pip install -qU unstructured

In [ ]:
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader

# 비구조화 CSV 로더 인스턴스 생성
loader = UnstructuredCSVLoader(file_path="/content/data/bostion_housing.csv", mode="elements")

# 문서 로드
docs = loader.load()

# 첫 번째 문서의 HTML 텍스트 메타데이터 출력
if docs and "text_as_html" in docs[0].metadata:
    print(docs[0].metadata["text_as_html"][:1000])
else:
    print("문서에 HTML 텍스트 메타데이터가 없습니다.")


## XML

In [ ]:
# XML 형식으로 변환된 데이터 저장
xml_data = ""

# 상위 10개 데이터 추출
for i, doc in enumerate(docs[:10]):  # 첫 10개 행만 처리
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitter_element = element.split(".")
        value = splitter_element[-1]  # 값
        col = ":".join(splitter_element[:-1])  # 컬럼 이름
        row_str += f"<{col}>{value}</{col}>"
    row_str += "</row>"
    xml_data += row_str

print(xml_data)


## 주택 가격에 영향을 주는 3가지 요소 정리

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

# 데이터 로드
data = pd.read_csv("/content/data/bostion_housing.csv")

# 독립 변수(X)와 종속 변수(y) 분리
X = data.drop(columns=["MEDV", "CAT.MEDV"])  # MEDV(주택 가격) 제외
y = data["MEDV"]  # MEDV를 목표 변수로 설정

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# 데이터 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 회귀 모델 학습
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# f-값 및 p-값 계산
f_values, p_values = f_regression(X_train_scaled, y_train)

# 각 변수의 중요도 정리
importance = pd.DataFrame({
    "Feature": X.columns,
    "F-Value": f_values,
    "P-Value": p_values,
    "Coefficient": model.coef_
}).sort_values(by="F-Value", ascending=False)

# 상위 3가지 변수 출력
top_3_features = importance.head(3)
print("주택 가격에 영향을 주는 상위 3가지 요소:")
print(top_3_features)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv("/content/data/bostion_housing.csv")

# 상위 3가지 특성 이름
top_3_features_names = ['RM', 'LSTAT', 'PTRATIO']  # 상위 3 특성 (예시)

# 상위 3가지 변수와 주택 가격 간의 상관 관계 시각화
correlation_matrix = data[top_3_features_names].copy()
correlation_matrix["MEDV"] = data["MEDV"]  # 주택 가격과 결합

# 상관계수 계산
correlation = correlation_matrix.corr()

# 히트맵으로 상관관계 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f", cbar=True)

plt.title('Correlation Heatmap: Top 3 Features vs MEDV')
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
data = pd.read_csv("/content/data/bostion_housing.csv")

# 독립 변수(X)와 종속 변수(y) 분리
X = data.drop(columns=["MEDV", "CAT.MEDV"])  # MEDV(주택 가격) 제외
y = data["MEDV"]  # MEDV를 목표 변수로 설정

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 회귀 모델 학습
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# f-값 및 p-값 계산
f_values, p_values = f_regression(X_train_scaled, y_train)

# 각 변수의 중요도 정리
importance = pd.DataFrame({
    "Feature": X.columns,
    "F-Value": f_values,
    "P-Value": p_values,
    "Coefficient": model.coef_
}).sort_values(by="F-Value", ascending=False)

# 상위 3가지 변수 출력
top_3_features = importance.head(3)

# 상위 3가지 요소에 대한 회귀선 그래프 그리기
top_3_features_names = top_3_features["Feature"]

# 원본 데이터에서 상위 3가지 변수 선택
top_3_data = data[top_3_features_names]

# 주택 가격과의 관계를 시각화
plt.figure(figsize=(12, 6))
for i, feature in enumerate(top_3_features_names):
    plt.subplot(1, 3, i + 1)
    sns.regplot(x=top_3_data[feature], y=y, line_kws={'color': 'red'}, scatter_kws={'s': 10})
    plt.title(f'{feature} vs MEDV')
    plt.xlabel(feature)
    plt.ylabel('MEDV')

plt.tight_layout()
plt.show()
